# AdaBoost classifieur from scratch

Dans ce notebook, vous allez coder votre propre algorithme de machine learning pour entraîner des modèles Adaboost avec des arbres de décision afin de résoudre des problèmes de classification. 

Le code que vous trouverez dans ce notebook est fonctionnelle pour créer un algorithme d'apprentissage automatique. Attention, ce code n'est pas optimis, il est uniquement à but pédagogique pour que vous compreniez bien ce qui se passe lorsque vous entraînez un algorithme de machine learning AdaBoost. Je vais vous accompagner pas à pas dans l'implémentation de cet algorithme.

Si vous voulez entraîner un modèle à but professionnel, je vous recommande d'utiliser le package [Sklearn](https://scikit-learn.org/stable/index.html) qui contient toutes les librairies essentiel pour vous aider. 

Merci de soutenir ce travail de création de contenus en vous abonnant à la chaîne youtube AIForYou et en mettant une étoile au répertoire github. 

## Importation des packages

In [1]:
# Packages de gestion des données
import numpy as np
import pandas as pd

# Packages de visualisation
import matplotlib.pyplot as plt

# Packages de modélisation
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

## Importation des données

In [2]:
data = load_breast_cancer()

x = data['data']
y = data['target']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

## AdaBoost

### Création de la classe

In [3]:
class adaBoostClassifier():
  def __init__(self, n_estimators=100, max_depth=1, random_state=None):
    self._n_estimators = n_estimators
    self._max_depth = max_depth
    self._random_state = random_state
    self._sample_weight = np.array([])
    self._weight_model = dict()
    self._model = dict()
    self._features = dict()

In [4]:
ada = adaBoostClassifier(n_estimators=20, random_state=123)
print(ada._n_estimators)
print(ada._max_depth)
print(ada._random_state)

20
1
123


### Fonction d'entraînement 

In [12]:
class adaBoostClassifier():
  def __init__(self, n_estimators=100, max_depth=1, random_state=None):
    self._n_estimators = n_estimators
    self._max_depth = max_depth
    self._random_state = random_state
    self._sample_weight = np.array([])
    self._weight_model = dict()
    self._model = dict()
    self._features = dict()

  def fit(self, x, y):

    # Initialisation des poids
    x_length = len(x)
    self._sample_weight = np.ones(x_length)/x_length


    np.random.seed(seed=9999)
    for est in range(self._n_estimators) :

      # Sélection aléatoire des indices colonnes et des indices exemples
      rand_index = np.random.randint(low=0, high=x.shape[0], size=round(x.shape[0]*0.8))
      rand_column = np.random.randint(low=0, high=x.shape[1], size=round(x.shape[1]*0.8))

      # Sélection aléatoire des colonnes et des exemples
      x_samp = x[rand_index, :]
      x_samp = x_samp[:, rand_column]
      y_samp = y[rand_index]
      weight_samp = self._sample_weight[rand_index]

      # Initialisation du modèle d'arbre de décision
      decision_tree_model = DecisionTreeClassifier(max_depth=self._max_depth, random_state=self._random_state)
      
      # Entraînement du modèle d'arbre de décision
      self._model[est] = decision_tree_model.fit(x_samp, y_samp, sample_weight=weight_samp)

      # Sauvegarde des colonnes utlisées pour l'arbre
      self._features[est] = rand_column

      # Prédition des données d'entraînement 
      y_pred = self._model[est].predict(x_samp)

      # Calcul de l'erreur total pondérée par le poids de chaque exemple
      total_error = np.sum((y_pred!=y_samp) * weight_samp)

      # Calcul du poids de l'arbre
      self._weight_model[est] = (1/2) * np.log((1-total_error)/(total_error))

      # Mise à jour des poids de l'arbre
      self._sample_weight[rand_index] = (y_pred!=y_samp) * self._sample_weight[rand_index] * np.exp(self._weight_model[est]) + (y_pred==y_samp) * self._sample_weight[rand_index] * np.exp(-self._weight_model[est])
  
      # Normalisation des poids  
      self._sample_weight /= np.sum(self._sample_weight)



In [23]:
ada = adaBoostClassifier(n_estimators=40, random_state=123)
ada.fit(x_train, y_train)

In [24]:
ada._model

{0: DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                        max_depth=1, max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort='deprecated',
                        random_state=123, splitter='best'),
 1: DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                        max_depth=1, max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort='deprecated',
                        random_state=123, splitter='best'),
 2: DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                        max_depth=1, max_features=None,

### Fonction de prédiction

In [29]:
class adaBoostClassifier():
  def __init__(self, n_estimators=100, max_depth=1, random_state=None):
    self._n_estimators = n_estimators
    self._max_depth = max_depth
    self._random_state = random_state
    self._sample_weight = np.array([])
    self._weight_model = dict()
    self._model = dict()
    self._features = dict()

  def fit(self, x, y):

    # Initialisation des poids
    x_length = len(x)
    self._sample_weight = np.ones(x_length)/x_length


    np.random.seed(seed=9999)
    for est in range(self._n_estimators) :

      # Sélection aléatoire des indices colonnes et des indices exemples

      rand_index = np.random.randint(low=0, high=x.shape[0], size=round(x.shape[0]*0.8))
      rand_column = np.random.randint(low=0, high=x.shape[1], size=round(x.shape[1]*0.8))

      # Sélection aléatoire des colonnes et des exemples
      x_samp = x[rand_index, :]
      x_samp = x_samp[:, rand_column]
      y_samp = y[rand_index]
      weight_samp = self._sample_weight[rand_index]

      # Initialisation du modèle d'arbre de décision
      decision_tree_model = DecisionTreeClassifier(max_depth=self._max_depth, random_state=self._random_state)
      
      # Entraînement du modèle d'arbre de décision
      self._model[est] = decision_tree_model.fit(x_samp, y_samp, sample_weight=weight_samp)

      # Sauvegarde des colonnes utlisées pour l'arbre
      self._features[est] = rand_column

      # Prédition des données d'entraînement 
      y_pred = self._model[est].predict(x_samp)

      # Calcul de l'erreur total pondérée par le poids de chaque exemple
      total_error = np.sum((y_pred!=y_samp) * weight_samp)

      # Calcul du poids de l'arbre
      self._weight_model[est] = (1/2) * np.log((1-total_error)/(total_error))

      # Mise à jour des poids de l'arbre
      self._sample_weight[rand_index] = (y_pred!=y_samp) * self._sample_weight[rand_index] * np.exp(self._weight_model[est]) + (y_pred==y_samp) * self._sample_weight[rand_index] * np.exp(-self._weight_model[est])
  
      # Normalisation des poids  
      self._sample_weight /= np.sum(self._sample_weight)


  def predict(self, x):

    # Initialiser le vecteur de prédiction
    pred = np.zeros(x.shape[0])

    # On ajoute à pred la prédiction de chaque arbre
    for i in range(self._n_estimators):
      pred += self._model[i].predict(x[:, self._features[i]]) * self._weight_model[i]

    # Normalisation des prédictions
    pred /= self._n_estimators

    # Passage d'une probabilité à une prédiction
    pred = np.where(pred >= 0.5, 1, 0)

    return pred

In [30]:
rf = adaBoostClassifier(n_estimators=40, random_state=123)
rf.fit(x_train, y_train)

In [31]:
pred = rf.predict(x_test)
np.mean(pred == y_test)

0.9840425531914894